In [1]:
#test 1

import numpy as np
import pandas as pd

eng1_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_eng_mix.csv"
spa1_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_spa_mix.csv"
eng2_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_eng_off.csv"
spa2_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_spa_off.csv"
eng3_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_eng_on.csv"
spa3_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_spa_on.csv"

df_spa1 = pd.read_csv(spa1_data)
df_eng1 = pd.read_csv(eng1_data)
df_spa2 = pd.read_csv(spa2_data)
df_eng2 = pd.read_csv(eng2_data)
df_spa3 = pd.read_csv(spa3_data)
df_eng3 = pd.read_csv(eng3_data)

df_eng = pd.concat([df_eng1, df_eng2, df_eng3], ignore_index=True)
df_spa = pd.concat([df_spa1, df_spa2, df_spa2], ignore_index=True)


#create dataframe
df_diff = pd.DataFrame({'file_name': df_eng.file_name, 'freq_eng': df_eng.freq_stdev_cent, 'freq_spa': df_spa.freq_stdev_cent,
                       'int_eng': df_eng.int_stdev, 'int_spa': df_spa.int_stdev, 'screen': df_eng['on/off']})

#removing anomalous phrases (low freq/int)
df_diff = df_diff.drop(df_diff[abs(df_diff.freq_eng) < 0.001].index)
df_diff = df_diff.drop(df_diff[abs(df_diff.freq_spa) < 0.001].index)
df_diff = df_diff.drop(df_diff[abs(df_diff.int_eng) < 0.001].index)
df_diff = df_diff.drop(df_diff[abs(df_diff.int_spa) < 0.001].index)
df_diff = df_diff.dropna()

#new columns
df_diff['diff_freq_in_cents'] = np.where(df_diff['freq_eng'] == df_diff['freq_spa'], 0, df_diff['freq_eng'] - df_diff['freq_spa'])
df_diff['diff_freq_proportion'] = ((df_diff.freq_eng / df_diff.freq_spa) - 1)
df_diff['winner_freq'] = np.where(df_diff['freq_eng'] > df_diff['freq_spa'], 'ENG', 'SPA')
df_diff['diff_int_db'] = np.where(df_diff['int_eng'] == df_diff['int_spa'], 0, df_diff['int_eng'] - df_diff['int_spa'])
df_diff['winner_int'] = np.where(df_diff['int_eng'] > df_diff['int_spa'], 'ENG', 'SPA')

df_diff

,file_name,freq_eng,freq_spa,int_eng,int_spa,screen,diff_freq_in_cents,diff_freq_proportion,winner_freq,diff_int_db,winner_int
0,2_10_eng_aligned_eng0007.1,1080.039480,1362.993247,3.162278,2.449490,mix,-282.953767,-0.207597,SPA,0.712788,ENG
1,2_10_eng_aligned_eng0010.1,679.326345,1081.016277,3.256158,3.214550,mix,-401.689932,-0.371585,SPA,0.041608,ENG
2,2_10_eng_aligned_eng0015.1,931.333018,157.762409,4.370858,5.736267,mix,773.570608,4.903390,ENG,-1.365410,SPA
3,2_10_eng_aligned_eng0023.2,1354.534989,701.537491,6.071335,1.940790,mix,652.997498,0.930809,ENG,4.130545,ENG
4,2_10_eng_aligned_eng0025.2,1156.174326,767.320924,2.828427,4.242641,mix,388.853403,0.506768,ENG,-1.414214,SPA
...,...,...,...,...,...,...,...,...,...,...,...
4003,s2_4_eng_aligned_eng0037.2,460.436609,421.818752,4.349329,4.041452,on,38.617856,0.091551,ENG,0.307878,ENG
4008,s2_4_eng_aligned_eng0044.2,321.420026,1426.649648,5.291503,3.236694,on,-1105.229622,-0.774703,SPA,2.054808,ENG
4012,s2_4_eng_aligned_eng0049.1,220.179324,1286.206661,2.629956,2.250926,on,-1066.027337,-0.828815,SPA,0.379030,ENG
4013,s2_4_eng_aligned_eng0050.1,1739.981176,1714.476989,2.516611,2.292686,on,25.504187,0.014876,ENG,0.223925,ENG


In [8]:
import pandas as pd
import pingouin as pg

aov = pg.anova(dv='diff_freq_in_cents', between=['screen'], data=df_diff,
             detailed=True)
aov2 = pg.anova(dv='diff_int_db', between=['screen'], data=df_diff,
             detailed=True)

print(aov)
print(aov2)

   Source            SS    DF             MS         F     p-unc       np2
0  screen  1.280899e+06     2  640449.674284  1.612789  0.199496  0.001029
1  Within  1.243341e+09  3131  397106.807113       NaN       NaN       NaN
   Source            SS    DF        MS         F     p-unc      np2
0  screen     17.534308     2  8.767154  2.743934  0.064471  0.00175
1  Within  10003.869132  3131  3.195104       NaN       NaN      NaN


In [9]:
#test 2 - normalized freq & int

eng1_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_mix_eng_new2.csv"
spa1_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_mix_spa_new2.csv"
eng2_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_on_eng_new2.csv"
spa2_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_on_spa_new2.csv"
eng3_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_off_eng_new2.csv"
spa3_data = r"C:/Users/erick/OneDrive/Desktop/pros_results/results_frequency_intensity_off_spa_new2.csv"

df_spa1 = pd.read_csv(spa1_data)
df_eng1 = pd.read_csv(eng1_data)
df_spa2 = pd.read_csv(spa2_data)
df_eng2 = pd.read_csv(eng2_data)
df_spa3 = pd.read_csv(spa3_data)
df_eng3 = pd.read_csv(eng3_data)

df_eng = pd.concat([df_eng1, df_eng2, df_eng3], ignore_index=True)
df_spa = pd.concat([df_spa1, df_spa2, df_spa2], ignore_index=True)

df_complete = pd.DataFrame({'file_name': df_eng.file_name, 'freq_eng': df_eng.freq_stdev, 'freq_spa': df_spa.freq_stdev,
                       'int_eng': df_eng.int_stdev, 'int_spa': df_spa.int_stdev, 'screen': df_eng['on/off']})
df_complete['freq_diff'] = df_complete['freq_eng'] - df_complete['freq_spa']
df_complete['int_diff'] = df_complete['int_eng'] - df_complete['int_spa']

df_complete = df_complete.drop(df_diff[abs(df_diff.freq_eng) < 0.001].index)
df_complete = df_complete.drop(df_diff[abs(df_diff.freq_spa) < 0.001].index)
df_complete = df_complete.drop(df_diff[abs(df_diff.int_eng) < 0.001].index)
df_complete = df_complete.drop(df_diff[abs(df_diff.int_spa) < 0.001].index)
df_complete = df_complete.dropna()

'''
df_complete = pd.concat([df_spa, df_eng], ignore_index=True)

df_complete = df_complete.drop(df_complete[abs(df_complete.freq_mean) < 0.001].index)
df_complete = df_complete.drop(df_complete[abs(df_complete.int_mean) < 0.001].index)
df_complete = df_complete.dropna(inplace=False)

#adding new columns
df_complete.rename(columns={'on/off':'screen'}, inplace=True)
'''

df_complete


,file_name,freq_eng,freq_spa,int_eng,int_spa,screen,freq_diff,int_diff
0,2_10_eng_aligned_eng0007.1,12.931464,3.924512,0.857981,0.624125,mix,9.006952,0.233857
1,2_10_eng_aligned_eng0010.1,2.901743,2.883189,0.954313,0.845847,mix,0.018554,0.108466
2,2_10_eng_aligned_eng0015.1,6.564598,1.597579,1.259905,1.566455,mix,4.967019,-0.306550
3,2_10_eng_aligned_eng0023.2,0.374934,1.867977,1.755032,0.506062,mix,-1.493043,1.248971
4,2_10_eng_aligned_eng0025.2,19.927895,10.208571,0.890672,1.185535,mix,9.719324,-0.294864
...,...,...,...,...,...,...,...,...
6972,s3_9_eng_aligned_eng0298.1,0.000000,1.890639,1.289833,1.356570,off,-1.890639,-0.066737
6973,s3_9_eng_aligned_eng0304.2,2.197952,6.746433,0.477264,1.298909,off,-4.548480,-0.821645
6974,s3_9_eng_aligned_eng0315.2,2.162879,1.757762,0.470196,0.689956,off,0.405117,-0.219760
6975,s3_9_eng_aligned_eng0316.5,2.073153,0.000000,0.302936,0.550330,off,2.073153,-0.247395


In [22]:
# mean difference for frequency stdev (in cents) and intensity (in db)

df_mix = df_complete.loc[df_complete['screen'] == 'mix']
df_on = df_complete.loc[df_complete['screen'] == 'on']
df_off = df_complete.loc[df_complete['screen'] == 'off']

dfs = [df_mix, df_on, df_off]

for dfx in dfs:
    diff_freq_sum = dfx['freq_diff'].sum(axis=0, skipna = True) 
    diff_int_sum = dfx['int_diff'].sum(axis=0, skipna = True)
    diff_freq_mean = diff_freq_sum/len(df_diff['freq_eng'])
    diff_int_mean = diff_int_sum/len(df_diff['int_eng'])
    
    print('frequency ', diff_freq_mean, ', ', 'intensity ', diff_int_mean)



frequency  0.41405283082131455 ,  intensity  0.05858174737747287
frequency  0.8856551293375878 ,  intensity  0.14461170505073387
frequency  0.46502591604243776 ,  intensity  -0.00023382195022335957


In [12]:
import pandas as pd
import pingouin as pg

df_complete.head()

aov = pg.anova(dv='freq_diff', between=['screen'], data=df_complete,
             detailed=True)
aov2 = pg.anova(dv='int_diff', between=['screen'], data=df_complete,
             detailed=True)

print(aov)
print(aov2)

   Source             SS    DF          MS         F     p-unc       np2
0  screen     279.543981     2  139.771990  4.845138  0.007893  0.001388
1  Within  201185.145283  6974   28.847884       NaN       NaN       NaN
   Source           SS    DF        MS          F         p-unc       np2
0  screen    12.234158     2  6.117079  16.920049  4.671829e-08  0.004829
1  Within  2521.299383  6974  0.361528        NaN           NaN       NaN
